In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import time

#Local modules
import models
import training
import loader
import tf_collections

In [2]:
regenerate = False #Don't relearn the Word2Vec embeddings
data = loader.Loader("corpora/", "./word_Vecs.npy", regenerate=regenerate) #Loads data and vanilla Word2Vec embeddings

train_prompts_int = data.train_prompts_int
train_answers_int = data.train_answers_int
valid_prompts_int = data.valid_prompts_int
valid_answers_int = data.valid_answers_int
vocab2int         = data.vocab2int
int2vocab         = data.int2vocab
unk_int           = data.unk_int
unk               = data.unk

In [3]:
time_string = time.strftime("%b%d_%H:%M:%S")
checkpoint_dir = os.path.join("checkpoints", time_string)
if not os.path.exists(checkpoint_dir): os.makedirs(checkpoint_dir)
checkpoint_best = str(checkpoint_dir) + "/" + "best_model.ckpt"
checkpoint_latest = str(checkpoint_dir) + "/" + "latest_model.ckpt"
sys.stderr.write("Writing all model files to {}\n".format(checkpoint_dir))

Writing all model files to checkpoints/Jul02_15:17:18


In [4]:
def append_eos(answers_int, eos_int):
        return [sequence+[eos_int] for sequence in answers_int]


In [5]:
def word_vecs_with_meta(wordVecs):
    """
    wordVecs - an np array of word embeddings
    Returns
            (np array with the metatoken embeddings appended, the index of the metatoken embedding)
    """
    embedding_size = wordVecs.shape[1] #Dynamically determine embedding size from loaded embedding file
    metatoken_embedding = np.zeros((1, embedding_size), dtype=wordVecs.dtype)
    wordVecsWithMeta = np.concatenate( (wordVecs, metatoken_embedding), axis=0 )
    return wordVecsWithMeta, wordVecsWithMeta.shape[0]-1

## VAD Appended Experiment

In [6]:
full_embeddings = data.load_vad("word_Vecs_VAD.npy", regenerate=regenerate)
(wordVecsWithMeta, metatoken) = word_vecs_with_meta(full_embeddings)
go_token = metatoken
eos_token = metatoken
pad_token = metatoken


train_answers_int = append_eos(train_answers_int, eos_token)
valid_answers_int = append_eos(valid_answers_int, eos_token)

#Just a class that makes it so we don't have to pass all 4 of these as separate parameters to functions
datasets = tf_collections.Datasets(train_prompts_int=train_prompts_int,
                        train_answers_int=train_answers_int,
                        valid_prompts_int=valid_prompts_int,
                        valid_answers_int=valid_answers_int
                )


tf.reset_default_graph()
data_placeholders = models.create_placeholders()
output_layer = tf.layers.Dense(len(wordVecsWithMeta),bias_initializer=tf.zeros_initializer(),
                               activation=tf.nn.relu)
model = models.VADAppended(data_placeholders, wordVecsWithMeta, go_token, eos_token,
                           output_layer=output_layer, affect_strength = 0.5)


xent_epochs = 12
train_feeds = {model.keep_prob: 0.75}
valid_feeds = {model.keep_prob: 1}

trainer = training.Trainer(checkpoint_best, checkpoint_latest, max_epochs=xent_epochs)
text_data = tf_collections.TextData(prompts_int2vocab=int2vocab,
                                answers_int2vocab=int2vocab,
                                unk_int=unk_int, eos_int=eos_token, pad_int=pad_token)

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        training.training_loop(sess, model, trainer, datasets, text_data,
                               train_feeds, valid_feeds, min_epochs_before_validation=2)

        affect_epochs = (trainer.epochs_completed // 4) + 1*(trainer.epochs_completed < 4)
        total_epochs = trainer.epochs_completed + affect_epochs
        train_feeds[model.train_affect] = True
        sys.stderr.write("Switching from cross-entropy to maximum affective content . . .\n")

        affect_trainer = training.Trainer(checkpoint_best, checkpoint_latest,
                                epochs_completed=trainer.epochs_completed,
                                max_epochs=total_epochs, saver=trainer.saver,
                                best_valid_cost = trainer.best_valid_cost)

        training.training_loop(sess, model, affect_trainer, datasets, text_data, train_feeds, valid_feeds)

Instructions for updating:
Use the retry module or similar alternatives.


ResourceExhaustedError: OOM when allocating tensor of shape [1024,256] and type float
	 [[Node: decoding/attention_wrapper/bahdanau_attention/query_layer/kernel/Adam/Initializer/zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [1024,256] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'decoding/attention_wrapper/bahdanau_attention/query_layer/kernel/Adam/Initializer/zeros', defined at:
  File "/opt/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start


## Aff2Vec Experiment

In [ ]:
counterfit=True #Use `False` for retrofitting
    
if counterfit:
        full_embeddings = data.load_counterfit("word_Vecs_counterfit_affect.npy",
                                                    "./w2v_counterfit_append_affect.bin",
                                                    regenerate=regenerate)
else:
        full_embeddings = data.load_counterfit("word_Vecs_retrofit_affect.npy",
                                                    "./w2v_counterfit_append_affect.bin",
                                                    regenerate=regenerate)
(wordVecsWithMeta, metatoken) = word_vecs_with_meta(full_embeddings)
go_token = metatoken
eos_token = metatoken
pad_token = metatoken


train_answers_int = append_eos(train_answers_int, eos_token)
valid_answers_int = append_eos(valid_answers_int, eos_token)
datasets = tf_collections.Datasets(train_prompts_int=train_prompts_int,
                        train_answers_int=train_answers_int,
                        valid_prompts_int=valid_prompts_int,
                        valid_answers_int=valid_answers_int
                )

tf.reset_default_graph()
data_placeholders = models.create_placeholders()
output_layer = tf.layers.Dense(len(wordVecsWithMeta),bias_initializer=tf.zeros_initializer(),activation=tf.nn.relu)
model = models.Aff2Vec(data_placeholders, wordVecsWithMeta, wordVecsWithMeta, go_token, eos_token,
                       output_layer=output_layer)

xent_epochs = 15
train_feeds = {model.keep_prob: 0.75}
valid_feeds = {model.keep_prob: 1}

trainer = training.Trainer(checkpoint_best, checkpoint_latest, max_epochs=xent_epochs, max_stalled_steps=2)
text_data = tf_collections.TextData(prompts_int2vocab=int2vocab, answers_int2vocab=int2vocab,
                                unk_int=unk_int, eos_int=eos_token, pad_int=pad_token)

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        training.training_loop(sess, model, trainer, datasets, text_data,
                               train_feeds, valid_feeds)